<a href="https://colab.research.google.com/github/zzhenxi/study-NLP-with-PyTorch/blob/main/%5BChapter3%5D_%EB%A0%88%EC%8A%A4%ED%86%A0%EB%9E%91_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/my_dataset/yelp_review/my_reviews_with_splits_lite.csv')

In [ ]:
df.head()

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train


# 데이터 백터화 클래스

## 파이토치 데이터셋 이해하기!

In [ ]:
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
  def __init__(self, review_df, vectorizer):
    self.review_df = review_df
    self._vecrotizer = vectorizer

    self.train_df = self.review_df[self.review_df.split =='train']
    self.train_size = len(self.train_df)

    self.val_df = self.review_df[self.review_df.split =='val']
    self.validation_size = len(self.val_df)

    self.test_df = self.review_df[self.review.split == 'test']
    self.test_size = len(self.test_df)

    self._lookup_dict = {'train': (self.train_df, self.train_size),
                         'val' : (self.val_df, self.validation_size),
                         'test' : (self.test_df, self.test_size)}
    
    self.set_split('train') # 이건 나중에 split의 매개변수를 우선 초기화 해준것?


  @classmethod
  def load_dataset_and_make_vectorizer(cls, review_csv):
    '''
    여기에서 처음에 ReviewDataset이라는 class의 매개변수를 넣어주는 행동을 하는 듯.
    cls는 곧 ReviewDataset라는 클래스이며, 그 클래스에 매개변수로 dataframe과 vectorizer를 넣어줌!
    '''
    review_df = pd.read_csv(review_csv)
    return cls(review_df, ReviewVectorizer.from_dataframe(review_df))

##### 이 함수는 굳이 왜 있는 걸까? vectorizer를 구현해보면 알듯..
  def get_vectorizer(self):
    return self._vectorizer
  
  def set_split(self, split='train'):
    self._target_split = split
    self._target_df, self._target_size = self._lookup_dict[split]

  def _len_(self):
    return self._target_size

  def _getitem_(self, index):
    '''
    index : 데이터 포인트의 인덱스 
    output -> 데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
    '''
    row = self._target_df.iloc[index]

    review_vector = \
      self._vectorizer.vectorize(row.review)
    
    rating_index = \
      self._vectorizer.rating_vocab.lookup_token(row.rating)

    return {'x_data': review_vector,
            'y_target': rating_index}

  def get_num_batches(self, batch_size):
    '''
    만들 수 있는 배치의 개수 반환
    '''
    return len(self) // batch_size


## Vocabulary, Vectorizer, DataLoader 클래스를 사용해 중요한 파이프라인을 수행

### Vocabulary

In [ ]:
class Vocabulary(object):
  '''매핑을 위해 텍스트를 처리하고 어휘사전을 만드는 클래스'''

  def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
    ''' 
    token_to_idx -> 기존 토큰-인덱스 매핑 딕셔너리
    add_unk (bool) -> UNK 토큰을 추가할지 지정하는 플래그 *여기서 UNK는 unknown을 뜻함
    unk_token (str) -> Vocabulary에 추가할 UNK 토큰
    '''

    if token_to_idx is None:
      token_to_idx = {}

      self._token_to_idx = token_to_idx

      self._idx_to_token = {idx:token
                            for token, idx in self._token_to_idx.items()} # token과 index위치 (키<->벨류) 바꿔줌
      
      self._add_unk = add_unk
      self._unk_token = unk_token
      
      self.unk_index = -1 # unk token의 인덱스는 -1이다.
      if add_unk:
        self.unk_index = self.add_token(unk_token)
      
      def to_serializable(self):
        '''직렬화할 수 있는 딕셔너리를 반환합니다''' # 그냥.. 딕셔너리를 반환한다는 얘기인가?
      return {'token_to_idx' : self._token_to_idx,
              'add_unk' : self._add_unk,
              'unk_token': self._unk_token}
      
      @classmethod
      def from_serializable(cls, contents):
        '''직렬화된 딕셔너리에서 Vocabulary 객체 만듦'''
        return cls(**contents) # **kwargs {키워드 : 특정값} 딕셔너리로 보냄 https://brunch.co.kr/@princox/180 참고

      def add_token(self, token):
        '''토큰을 기반으로 매핑 딕셔너리를 업데이트 합니다. 
        매개변수 : 
        token (str) : Vocabulary에 추가할 토큰
        반환값 : 
        index (int) : 토큰에 상응하는 정수 
        '''
        if token in self._token_to_idx:
          index = self._token_to_idx[token]
        else:
          index = len(self._token_to_idx)
          self._token_to_idx[token] = index
          self._idx_to_token[index] = token
        return index

        





In [ ]:
a = {1:'one', 2:'two', 3:'three'}

In [ ]:
a= {idx:num
    for num, idx in a.items()}

In [ ]:
a

{'one': 1, 'three': 3, 'two': 2}